In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import pandas as pd

def crear_navegador():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Ejecutar sin interfaz gráfica
    options.add_argument("--disable-gpu")
    options.add_argument("--blink-settings=imagesEnabled=false")
    options.add_argument("--disable-extensions")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def extraer_ofertas_manfred():
    browser = crear_navegador()
    url = "https://www.getmanfred.com/ofertas-empleo?onlyActive=false&currency=%E2%82%AC"
    browser.get(url)
    browser.implicitly_wait(10)
    
    ofertas = browser.find_elements(By.CSS_SELECTOR, 'a[href^="/ofertas-empleo/"]')
    urls_ofertas = [oferta.get_attribute("href") for oferta in ofertas]
    
    todas_ofertas_extraidas = []
    MAX_OFERTAS_POR_SESION = 100
    ofertas_procesadas = 0
    
    for i, oferta_url in enumerate(urls_ofertas):
        try:
            if ofertas_procesadas == MAX_OFERTAS_POR_SESION:
                browser.quit()
                browser = crear_navegador()
                ofertas_procesadas = 0
            
            browser.get(oferta_url)
            WebDriverWait(browser, 15).until(EC.presence_of_element_located((By.CLASS_NAME, "sc-f61a956f-0")))
            
            url_actual = browser.current_url
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            oferta_titulo = browser.find_element(By.CLASS_NAME, "sc-f61a956f-0").text if browser.find_elements(By.CLASS_NAME, "sc-f61a956f-0") else "No disponible"
            
            oferta_info_texto = browser.find_elements(By.CLASS_NAME, "sc-6d3685c2-5")
            textos = [elem.text for elem in oferta_info_texto]
            oferta_lugar = browser.find_elements(By.CLASS_NAME, "sc-6d3685c2-2")
            datos = [elem.text for elem in oferta_lugar]
            diccionario_oferta01 = dict(zip(textos, datos))
            
            oferta_info_texto02 = browser.find_elements(By.CLASS_NAME, "sc-c51cd5df-3")
            textos02 = [elem.text for elem in oferta_info_texto02]
            oferta_lugar02 = browser.find_elements(By.CLASS_NAME, "sc-c51cd5df-4")
            datos02 = [elem.text for elem in oferta_lugar02]
            diccionario_oferta02 = dict(zip(textos02, datos02))
            
            tecnologias = browser.find_elements(By.CLASS_NAME, "sc-e4487fe1-2")
            tecnologias_info = [elem.text for elem in tecnologias]
            niveles = browser.find_elements(By.CLASS_NAME, "sc-e4487fe1-5")
            niveles_tecnologia = [elem.text for elem in niveles]
            diccionario_tecnologias = dict(zip(tecnologias_info, niveles_tecnologia))
            
            otras_habilidades = browser.find_elements(By.CLASS_NAME, "sc-22385303-2")
            otras_habilidades_info = [elem.text for elem in otras_habilidades]
            
            oferta_activa = browser.find_element(By.CLASS_NAME, "sc-81fb91b0-11").text if browser.find_elements(By.CLASS_NAME, "sc-81fb91b0-11") else "Activa"
            
            oferta_info = {
                "url": url_actual,
                "fecha_extraccion": timestamp,
                "titulo": oferta_titulo,
                "oferta_activada": oferta_activa,
                "detalles_1": diccionario_oferta01,
                "detalles_2": diccionario_oferta02,
                "tecnologias": {"Tecnologías": diccionario_tecnologias},
                "habilidades": {"Otras habilidades": otras_habilidades_info}
            }
            
            todas_ofertas_extraidas.append(oferta_info)
            ofertas_procesadas += 1
            print(f"Oferta {i+1} guardada correctamente.")
        
        except Exception as e:
            print(f"Error con la oferta {i+1}: {e}")
    
    browser.quit()
    
    df = pd.json_normalize(todas_ofertas_extraidas, sep='_')
    df.to_csv(f'manfred{timestamp}.csv', index=False)
    print("Datos guardados en csv")

In [3]:
extraer_ofertas_manfred()

Oferta 1 guardada correctamente.
Oferta 2 guardada correctamente.
Oferta 3 guardada correctamente.
Oferta 4 guardada correctamente.
Oferta 5 guardada correctamente.
Oferta 6 guardada correctamente.
Oferta 7 guardada correctamente.
Oferta 8 guardada correctamente.
Oferta 9 guardada correctamente.
Oferta 10 guardada correctamente.
Oferta 11 guardada correctamente.
Oferta 12 guardada correctamente.
Oferta 13 guardada correctamente.
Oferta 14 guardada correctamente.
Oferta 15 guardada correctamente.
Oferta 16 guardada correctamente.
Oferta 17 guardada correctamente.
Oferta 18 guardada correctamente.
Oferta 19 guardada correctamente.
Oferta 20 guardada correctamente.
Oferta 21 guardada correctamente.
Oferta 22 guardada correctamente.
Oferta 23 guardada correctamente.
Oferta 24 guardada correctamente.
Oferta 25 guardada correctamente.
Oferta 26 guardada correctamente.
Oferta 27 guardada correctamente.
Oferta 28 guardada correctamente.
Oferta 29 guardada correctamente.
Oferta 30 guardada corr